In [3]:
"""
SCRIPT INICIALIZACIÓN - limpieza_datos.ipynb
Ejecutar esta celda primero después de reiniciar el kernel
"""

import pandas as pd
import numpy as np
import yfinance as yf
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("INICIALIZANDO NOTEBOOK LIMPIEZA DE DATOS...")
print("=" * 60)

# ===================================================================
# VERIFICAR/CREAR ESTRUCTURA DE CARPETAS
# ===================================================================
print("\n1. VERIFICANDO ESTRUCTURA DE CARPETAS...")

carpetas = [
    'data',
    'data/raw',
    'data/raw/crypto', 
    'data/raw/stocks',
    'data/processed',
    'reports'
]

for carpeta in carpetas:
    os.makedirs(carpeta, exist_ok=True)
    
print(f"   ✅ {len(carpetas)} carpetas verificadas/creadas")

# ===================================================================
# CONFIGURACIÓN
# ===================================================================
print("\n2. CONFIGURACIÓN...")

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("   ✅ Pandas configurado")
print("   ✅ yfinance disponible")

# ===================================================================
# RESUMEN
# ===================================================================
print("\n" + "=" * 60)
print("LISTO PARA LIMPIEZA DE DATOS")
print("=" * 60)
print("Este notebook genera datos desde cero:")
print("  - Descarga de yfinance")
print("  - Procesamiento de archivos Kaggle")
print("  - Limpieza y transformación")
print("  - Guardado en data/processed/")
print("\nPuedes ejecutar las siguientes celdas para generar los datos")

INICIALIZANDO NOTEBOOK LIMPIEZA DE DATOS...

1. VERIFICANDO ESTRUCTURA DE CARPETAS...
   ✅ 6 carpetas verificadas/creadas

2. CONFIGURACIÓN...
   ✅ Pandas configurado
   ✅ yfinance disponible

LISTO PARA LIMPIEZA DE DATOS
Este notebook genera datos desde cero:
  - Descarga de yfinance
  - Procesamiento de archivos Kaggle
  - Limpieza y transformación
  - Guardado en data/processed/

Puedes ejecutar las siguientes celdas para generar los datos


In [5]:
# ===================================================================
# PROYECTO: Análisis Comparativo Criptomonedas vs Acciones Tecnológicas (2012-2024)
# NOTEBOOK: 01 - Limpieza y Transformación de Datos 
# AUTOR: [Sergio Cano]
# FECHA: [Diciembre 2024]
# DATASET: 50K+ filas, 17 activos, 13 años datos históricos
# ===================================================================

# Importar librerías necesarias
import pandas as pd
import numpy as np
import yfinance as yf
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 1. Crear estructura de carpetas
print("📁 Creando estructura de carpetas...")
directories = [
    'data',
    'data/raw',
    'data/raw/crypto', 
    'data/raw/stocks',
    'data/processed',
    'reports'
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"   ✅ {directory}")

# 2. Definir activos y configuración
print(f"\n🎯 Definiendo activos para descarga...")
crypto_symbols = {
    'bitcoin': 'BTC-USD',
    'ethereum': 'ETH-USD', 
    'binancecoin': 'BNB-USD',
    'cardano': 'ADA-USD',
    'solana': 'SOL-USD',
    'dogecoin': 'DOGE-USD',
    'polkadot': 'DOT-USD'
}

stock_symbols = {
    'aapl': 'AAPL',
    'tsla': 'TSLA',
    'msft': 'MSFT', 
    'amzn': 'AMZN',
    'googl': 'GOOGL',
    'nvda': 'NVDA',
    'meta': 'META',
    'intc': 'INTC',
    'amd': 'AMD',
    'qcom': 'QCOM'
}

# Fechas: 13 años para superar 50k filas
start_date = '2012-01-01'
end_date = '2024-12-31'

print(f"💰 Criptomonedas: {len(crypto_symbols)}")
print(f"📈 Acciones: {len(stock_symbols)}")
print(f"📅 Período: {start_date} a {end_date}")

# 3. Función de descarga 
def download_asset(name, symbol, asset_type, start_date, end_date):
    """Descarga un activo con manejo de errores"""
    try:
        print(f"🔄 {name.upper():<12} ({symbol})...")
        
        data = yf.download(symbol, start=start_date, end=end_date, progress=False)
        
        if data.empty:
            print(f"   ❌ Sin datos")
            return None
        
        # Arreglar MultiIndex si existe
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)
        
        # Resetear índice y estandarizar columnas
        data = data.reset_index()
        data.columns = data.columns.str.lower().str.replace(' ', '_')
        
        # Limpiar datos básicos
        data = data.dropna(subset=['close'])
        data = data[data['close'] > 0]
        
        # Añadir metadatos
        data['asset_name'] = name.upper()
        data['asset_type'] = asset_type
        data['asset_class'] = asset_type
        data['symbol'] = symbol
        
        print(f"   ✅ {len(data):,} registros ({data['date'].min().date()} a {data['date'].max().date()})")
        return data
        
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        return None

print(f"\n🔄 INICIANDO DESCARGA MASIVA...")


📁 Creando estructura de carpetas...
   ✅ data
   ✅ data/raw
   ✅ data/raw/crypto
   ✅ data/raw/stocks
   ✅ data/processed
   ✅ reports

🎯 Definiendo activos para descarga...
💰 Criptomonedas: 7
📈 Acciones: 10
📅 Período: 2012-01-01 a 2024-12-31

🔄 INICIANDO DESCARGA MASIVA...


In [6]:
print("📊 DESCARGANDO TODOS LOS ACTIVOS")
print("=" * 60)

# Descargar criptomonedas
print("💰 DESCARGANDO CRIPTOMONEDAS:")
crypto_datasets = {}
for name, symbol in crypto_symbols.items():
    data = download_asset(name, symbol, 'crypto', start_date, end_date)
    if data is not None:
        crypto_datasets[name] = data

print(f"\n📈 DESCARGANDO ACCIONES:")
stock_datasets = {}
for name, symbol in stock_symbols.items():
    data = download_asset(name, symbol, 'stock', start_date, end_date)
    if data is not None:
        stock_datasets[name] = data

# Resumen de descarga
print(f"\n📊 RESUMEN DE DESCARGA:")
print(f"   💰 Criptos exitosas: {len(crypto_datasets)}/{len(crypto_symbols)}")
print(f"   📈 Stocks exitosas: {len(stock_datasets)}/{len(stock_symbols)}")

crypto_total = sum(len(df) for df in crypto_datasets.values())
stock_total = sum(len(df) for df in stock_datasets.values())

print(f"   💰 Filas crypto: {crypto_total:,}")
print(f"   📈 Filas stocks: {stock_total:,}")
print(f"   🎯 TOTAL: {crypto_total + stock_total:,} filas")

if crypto_total + stock_total >= 50000:
    print(f"   ✅ CUMPLE REQUISITO 50k+ filas")
else:
    print(f"   ⚠️ Faltan {50000 - (crypto_total + stock_total):,} filas")

📊 DESCARGANDO TODOS LOS ACTIVOS
💰 DESCARGANDO CRIPTOMONEDAS:
🔄 BITCOIN      (BTC-USD)...
   ✅ 3,758 registros (2014-09-17 a 2024-12-30)
🔄 ETHEREUM     (ETH-USD)...
   ✅ 2,609 registros (2017-11-09 a 2024-12-30)
🔄 BINANCECOIN  (BNB-USD)...
   ✅ 2,609 registros (2017-11-09 a 2024-12-30)
🔄 CARDANO      (ADA-USD)...
   ✅ 2,609 registros (2017-11-09 a 2024-12-30)
🔄 SOLANA       (SOL-USD)...
   ✅ 1,726 registros (2020-04-10 a 2024-12-30)
🔄 DOGECOIN     (DOGE-USD)...
   ✅ 2,609 registros (2017-11-09 a 2024-12-30)
🔄 POLKADOT     (DOT-USD)...
   ✅ 1,594 registros (2020-08-20 a 2024-12-30)

📈 DESCARGANDO ACCIONES:
🔄 AAPL         (AAPL)...
   ✅ 3,269 registros (2012-01-03 a 2024-12-30)
🔄 TSLA         (TSLA)...
   ✅ 3,269 registros (2012-01-03 a 2024-12-30)
🔄 MSFT         (MSFT)...
   ✅ 3,269 registros (2012-01-03 a 2024-12-30)
🔄 AMZN         (AMZN)...
   ✅ 3,269 registros (2012-01-03 a 2024-12-30)
🔄 GOOGL        (GOOGL)...
   ✅ 3,269 registros (2012-01-03 a 2024-12-30)
🔄 NVDA         (NVDA)...
  

In [8]:
print("\n🧹 LIMPIEZA Y PROCESAMIENTO FINAL")
print("=" * 60)

def process_for_daily_frequency(datasets_dict, asset_type):
    """Procesa datasets para frecuencia diaria completa"""
    print(f"🔄 Procesando {asset_type}...")
    
    if not datasets_dict:
        return []
    
    # Obtener rango completo de fechas
    all_dates = []
    for df in datasets_dict.values():
        all_dates.extend(df['date'].tolist())
    
    min_date = min(all_dates)
    max_date = max(all_dates)
    complete_date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    
    processed_datasets = []
    
    for name, df in datasets_dict.items():
        df_copy = df.copy().set_index('date').sort_index()
        
        if asset_type == 'stock':
            # Para stocks: expandir a todos los días y forward fill
            df_expanded = df_copy.reindex(complete_date_range)
            
            # Forward fill precios (mantiene precio del viernes en fin de semana)
            price_cols = ['open', 'high', 'low', 'close']
            for col in price_cols:
                if col in df_expanded.columns:
                    df_expanded[col] = df_expanded[col].fillna(method='ffill')
            
            # Volumen = último volumen conocido (o 0)
            if 'volume' in df_expanded.columns:
                df_expanded['volume'] = df_expanded['volume'].fillna(method='ffill').fillna(0)
            
            df_processed = df_expanded.reset_index().rename(columns={'index': 'date'})
            
        else:
            # Para crypto: solo limpiar NaNs (ya tiene frecuencia diaria)
            for col in ['open', 'high', 'low', 'close', 'volume']:
                if col in df_copy.columns:
                    df_copy[col] = df_copy[col].fillna(method='ffill').fillna(method='bfill')
            
            df_processed = df_copy.reset_index()
        
        # Rellenar metadatos
        df_processed['asset_name'] = name.upper()
        df_processed['asset_type'] = asset_type
        df_processed['asset_class'] = asset_type
        
        # Eliminar filas sin precio
        df_processed = df_processed.dropna(subset=['close'])
        
        processed_datasets.append(df_processed)
        print(f"   ✅ {name.upper()}: {len(df_processed):,} filas")
    
    return processed_datasets

# Procesar crypto y stocks
print("💰 PROCESANDO CRIPTOMONEDAS:")
crypto_processed = process_for_daily_frequency(crypto_datasets, 'crypto')

print("\n📈 PROCESANDO ACCIONES (Forward Fill completo):")
stock_processed = process_for_daily_frequency(stock_datasets, 'stock')

# Combinar todo
print(f"\n🔗 COMBINANDO DATASETS:")
all_datasets = crypto_processed + stock_processed
final_dataset = pd.concat(all_datasets, ignore_index=True, sort=False)
final_dataset = final_dataset.sort_values(['asset_name', 'date']).reset_index(drop=True)

print(f"   📏 Dataset final: {final_dataset.shape}")
print(f"   📅 Período: {final_dataset['date'].min().date()} a {final_dataset['date'].max().date()}")
print(f"   🎯 Activos: {final_dataset['asset_name'].nunique()}")

# Verificar NaNs
total_nans = final_dataset.isnull().sum().sum()
print(f"   ❓ NaNs totales: {total_nans:,}")

# Guardar dataset
output_file = '../../data/processed/combined_crypto_stock_data.csv'
final_dataset.to_csv(output_file, index=False)
file_size = os.path.getsize(output_file) / 1024 / 1024

print(f"\n💾 DATASET GUARDADO:")
print(f"   📄 {output_file}")
print(f"   💽 Tamaño: {file_size:.1f} MB")

print(f"\n🎉 ¡PROCESO COMPLETADO!")
print(f"✅ Dataset listo con {final_dataset.shape[0]:,} filas y {final_dataset.shape[1]} columnas")


🧹 LIMPIEZA Y PROCESAMIENTO FINAL
💰 PROCESANDO CRIPTOMONEDAS:
🔄 Procesando crypto...
   ✅ BITCOIN: 3,758 filas
   ✅ ETHEREUM: 2,609 filas
   ✅ BINANCECOIN: 2,609 filas
   ✅ CARDANO: 2,609 filas
   ✅ SOLANA: 1,726 filas
   ✅ DOGECOIN: 2,609 filas
   ✅ POLKADOT: 1,594 filas

📈 PROCESANDO ACCIONES (Forward Fill completo):
🔄 Procesando stock...
   ✅ AAPL: 4,746 filas
   ✅ TSLA: 4,746 filas
   ✅ MSFT: 4,746 filas
   ✅ AMZN: 4,746 filas
   ✅ GOOGL: 4,746 filas
   ✅ NVDA: 4,746 filas
   ✅ META: 4,610 filas
   ✅ INTC: 4,746 filas
   ✅ AMD: 4,746 filas
   ✅ QCOM: 4,746 filas

🔗 COMBINANDO DATASETS:
   📏 Dataset final: (64838, 10)
   📅 Período: 2012-01-03 a 2024-12-30
   🎯 Activos: 17
   ❓ NaNs totales: 14,729

💾 DATASET GUARDADO:
   📄 ../../data/processed/combined_crypto_stock_data.csv
   💽 Tamaño: 7.4 MB

🎉 ¡PROCESO COMPLETADO!
✅ Dataset listo con 64,838 filas y 10 columnas


In [23]:
print("📊 DATASET FINAL - INSPECCIÓN COMPLETA")
print("=" * 70)

# Configurar pandas para ver todo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', None)

print(f"📏 INFORMACIÓN GENERAL:")
print(f"   Forma: {final_dataset.shape}")
print(f"   Período: {final_dataset['date'].min().date()} a {final_dataset['date'].max().date()}")
print(f"   Años: {(final_dataset['date'].max() - final_dataset['date'].min()).days / 365.25:.1f}")

print(f"\n📋 COLUMNAS FINALES:")
for i, col in enumerate(final_dataset.columns, 1):
    dtype = str(final_dataset[col].dtype)
    nan_count = final_dataset[col].isnull().sum()
    unique_vals = final_dataset[col].nunique()
    print(f"   {i:2d}. {col:<15} | {dtype:<12} | NaN: {nan_count:>6} | Únicos: {unique_vals:>8,}")

print(f"\n📊 PRIMERAS 10 FILAS (TODAS LAS COLUMNAS):")
display(final_dataset.head(10))

print(f"\n🎯 RESUMEN POR ACTIVO:")
asset_summary = final_dataset.groupby(['asset_name', 'asset_class']).agg({
    'date': ['count', 'min', 'max'],
    'close': ['min', 'max']
}).round(2)

# Simplificar columnas
asset_summary.columns = ['Filas', 'Inicio', 'Fin', 'Min_Precio', 'Max_Precio']
asset_summary = asset_summary.reset_index()

for _, row in asset_summary.iterrows():
    años = (pd.to_datetime(row['Fin']) - pd.to_datetime(row['Inicio'])).days / 365.25
    print(f"   {row['asset_name']:<12} ({row['asset_class']}): {row['Filas']:>6} filas | {años:>4.1f} años | ${row['Min_Precio']:>8,.2f}-${row['Max_Precio']:>10,.2f}")

print(f"\n✅ DATASET COMPLETO LISTO PARA ANÁLISIS")


📊 DATASET FINAL - INSPECCIÓN COMPLETA
📏 INFORMACIÓN GENERAL:
   Forma: (50109, 10)
   Período: 2012-01-03 a 2024-12-30
   Años: 13.0

📋 COLUMNAS FINALES:
    1. date            | datetime64[ns] | NaN:      0 | Únicos:    4,438
    2. close           | float64      | NaN:      0 | Únicos:   47,609
    3. high            | float64      | NaN:      0 | Únicos:   48,186
    4. low             | float64      | NaN:      0 | Únicos:   48,190
    5. open            | float64      | NaN:      0 | Únicos:   48,173
    6. volume          | float64      | NaN:      0 | Únicos:   49,359
    7. asset_name      | object       | NaN:      0 | Únicos:       17
    8. asset_type      | object       | NaN:      0 | Únicos:        2
    9. asset_class     | object       | NaN:      0 | Únicos:        2
   10. symbol          | object       | NaN:      0 | Únicos:       17

📊 PRIMERAS 10 FILAS (TODAS LAS COLUMNAS):


Price,date,close,high,low,open,volume,asset_name,asset_type,asset_class,symbol
0,2012-01-03,12.345174,12.383300,12.278229,12.290238,302220800.0,Apple,stock,stock,AAPL
1,2012-01-04,12.411519,12.448745,12.286636,12.308250,260022000.0,Apple,stock,stock,AAPL
2,2012-01-05,12.549310,12.564920,12.388401,12.456848,271269600.0,Apple,stock,stock,AAPL
3,2012-01-06,12.680497,12.691004,12.585033,12.601544,318292800.0,Apple,stock,stock,AAPL
6,2012-01-09,12.660385,12.841106,12.648977,12.773562,394024400.0,Apple,stock,stock,AAPL
7,2012-01-10,12.705711,12.788567,12.653476,12.785865,258196400.0,Apple,stock,stock,AAPL
8,2012-01-11,12.684997,12.694003,12.587732,12.688899,215084800.0,Apple,stock,stock,AAPL
9,2012-01-12,12.650176,12.695506,12.570923,12.676894,212587200.0,Apple,stock,stock,AAPL
10,2012-01-13,12.602749,12.621962,12.568226,12.599448,226021600.0,Apple,stock,stock,AAPL
14,2012-01-17,12.749546,12.788272,12.697311,12.734536,242897200.0,Apple,stock,stock,AAPL



🎯 RESUMEN POR ACTIVO:
   Advanced Micro Devices (stock):   3269 filas | 13.0 años | $    1.62-$    211.38
   Amazon       (stock):   3269 filas | 13.0 años | $    8.80-$    232.93
   Apple        (stock):   3269 filas | 13.0 años | $   11.90-$    258.10
   BINANCECOIN  (crypto):   2609 filas |  7.1 años | $    1.51-$    750.27
   BITCOIN      (crypto):   3758 filas | 10.3 años | $  178.10-$106,140.60
   CARDANO      (crypto):   2609 filas |  7.1 años | $    0.02-$      2.97
   DOGECOIN     (crypto):   2609 filas |  7.1 años | $    0.00-$      0.68
   ETHEREUM     (crypto):   2609 filas |  7.1 años | $   84.31-$  4,812.09
   Google       (stock):   3269 filas | 13.0 años | $   13.89-$    196.02
   Intel        (stock):   3269 filas | 13.0 años | $   13.84-$     62.08
   Meta         (stock):   3174 filas | 12.6 años | $   17.62-$    630.70
   Microsoft    (stock):   3269 filas | 13.0 años | $   21.00-$    463.24
   NVIDIA       (stock):   3269 filas | 13.0 años | $    0.26-$    148.84


In [24]:
# Verificar si tenemos el dataset cargado
if 'final_dataset' in globals() and final_dataset is not None:
    print("🔍 AUDITORÍA DE NaN - COLUMNA SYMBOL")
    print("=" * 60)
    
    # Información general sobre symbol
    print(f"📊 COLUMNA 'SYMBOL' - ANÁLISIS COMPLETO:")
    if 'symbol' in final_dataset.columns:
        symbol_col = final_dataset['symbol']
        
        print(f"   Total registros: {len(symbol_col):,}")
        print(f"   NaN count: {symbol_col.isnull().sum():,}")
        print(f"   NaN percentage: {(symbol_col.isnull().sum() / len(symbol_col)) * 100:.2f}%")
        print(f"   Valores únicos: {symbol_col.nunique()}")
        print(f"   Valores únicos (sin NaN): {symbol_col.dropna().nunique()}")
        
        # Ver valores únicos
        print(f"\n📋 VALORES ÚNICOS EN SYMBOL:")
        unique_symbols = symbol_col.dropna().unique()
        for i, sym in enumerate(sorted(unique_symbols), 1):
            count = (symbol_col == sym).sum()
            print(f"   {i:2d}. {sym:<10} ({count:,} registros)")
        
        # IDENTIFICAR EXACTAMENTE DÓNDE ESTÁN LOS NaN
        nan_rows = final_dataset[final_dataset['symbol'].isnull()]
        
        if len(nan_rows) > 0:
            print(f"\n🚨 NaN EN SYMBOL - {len(nan_rows):,} REGISTROS:")
            print(f"   Por activo:")
            nan_by_asset = nan_rows['asset_name'].value_counts()
            for asset, count in nan_by_asset.items():
                total_asset = (final_dataset['asset_name'] == asset).sum()
                pct = (count / total_asset) * 100
                print(f"     {asset:<12}: {count:>6,}/{total_asset:,} ({pct:>5.1f}%)")
            
            print(f"\n📊 MUESTRA DE REGISTROS CON SYMBOL NaN:")
            sample_nan = nan_rows[['date', 'asset_name', 'asset_class', 'symbol', 'close']].head(10)
            display(sample_nan)
            
        else:
            print(f"   ✅ NO HAY NaN EN SYMBOL")
    
    else:
        print(f"   ❌ COLUMNA 'SYMBOL' NO EXISTE")
    
else:
    print("❌ DATASET NO CARGADO")
    print("💡 Ejecuta primero las celdas de descarga de datos")

🔍 AUDITORÍA DE NaN - COLUMNA SYMBOL
📊 COLUMNA 'SYMBOL' - ANÁLISIS COMPLETO:
   Total registros: 50,109
   NaN count: 0
   NaN percentage: 0.00%
   Valores únicos: 17
   Valores únicos (sin NaN): 17

📋 VALORES ÚNICOS EN SYMBOL:
    1. AAPL       (3,269 registros)
    2. ADA-USD    (2,609 registros)
    3. AMD        (3,269 registros)
    4. AMZN       (3,269 registros)
    5. BNB-USD    (2,609 registros)
    6. BTC-USD    (3,758 registros)
    7. DOGE-USD   (2,609 registros)
    8. DOT-USD    (1,594 registros)
    9. ETH-USD    (2,609 registros)
   10. GOOGL      (3,269 registros)
   11. INTC       (3,269 registros)
   12. META       (3,174 registros)
   13. MSFT       (3,269 registros)
   14. NVDA       (3,269 registros)
   15. QCOM       (3,269 registros)
   16. SOL-USD    (1,726 registros)
   17. TSLA       (3,269 registros)
   ✅ NO HAY NaN EN SYMBOL


In [10]:
if 'final_dataset' in globals() and final_dataset is not None:
    print("\n🔍 AUDITORÍA COMPLETA DE NaN - TODAS LAS COLUMNAS")
    print("=" * 70)
    
    # Análisis detallado por columna
    print("📊 ESTADO DE NaN POR COLUMNA:")
    print("-" * 100)
    print(f"{'#':<3} {'Columna':<15} {'Total_NaN':<10} {'NaN_%':<8} {'Estado':<15} {'Acción_Tomada':<20}")
    print("-" * 100)
    
    nan_analysis = []
    
    for i, col in enumerate(final_dataset.columns, 1):
        nan_count = final_dataset[col].isnull().sum()
        nan_pct = (nan_count / len(final_dataset)) * 100
        
        # Determinar estado y acción tomada
        if nan_count == 0:
            estado = "✅ LIMPIO"
            accion = "Sin NaN originales"
        elif nan_pct < 1:
            estado = "🟡 POCOS NaN"
            accion = "Forward/Backward fill"
        elif nan_pct < 10:
            estado = "🟠 ALGUNOS NaN"
            accion = "Forward/Backward fill"
        elif nan_pct < 50:
            estado = "🔴 MUCHOS NaN"
            accion = "¿Requiere revisión?"
        else:
            estado = "🚨 CRÍTICO"
            accion = "¿Columna problemática?"
        
        print(f"{i:<3} {col:<15} {nan_count:<10,} {nan_pct:<8.2f} {estado:<15} {accion:<20}")
        
        nan_analysis.append({
            'columna': col,
            'nan_count': nan_count,
            'nan_pct': nan_pct,
            'estado': estado
        })
    
    # Resumen por estado
    print(f"\n📊 RESUMEN POR ESTADO:")
    estados = {}
    for analysis in nan_analysis:
        estado = analysis['estado']
        estados[estado] = estados.get(estado, 0) + 1
    
    for estado, count in estados.items():
        print(f"   {estado}: {count} columnas")
    
    # Columnas con más NaN
    problematic_cols = [a for a in nan_analysis if a['nan_count'] > 0]
    
    if problematic_cols:
        print(f"\n⚠️ COLUMNAS CON NaN RESTANTES:")
        for col_info in sorted(problematic_cols, key=lambda x: x['nan_count'], reverse=True):
            print(f"   {col_info['columna']}: {col_info['nan_count']:,} NaN ({col_info['nan_pct']:.2f}%)")
    
else:
    print("❌ Dataset no disponible")


🔍 AUDITORÍA COMPLETA DE NaN - TODAS LAS COLUMNAS
📊 ESTADO DE NaN POR COLUMNA:
----------------------------------------------------------------------------------------------------
#   Columna         Total_NaN  NaN_%    Estado          Acción_Tomada       
----------------------------------------------------------------------------------------------------
1   date            0          0.00     ✅ LIMPIO        Sin NaN originales  
2   close           0          0.00     ✅ LIMPIO        Sin NaN originales  
3   high            0          0.00     ✅ LIMPIO        Sin NaN originales  
4   low             0          0.00     ✅ LIMPIO        Sin NaN originales  
5   open            0          0.00     ✅ LIMPIO        Sin NaN originales  
6   volume          0          0.00     ✅ LIMPIO        Sin NaN originales  
7   asset_name      0          0.00     ✅ LIMPIO        Sin NaN originales  
8   asset_type      0          0.00     ✅ LIMPIO        Sin NaN originales  
9   asset_class     0     

In [11]:
if 'final_dataset' in globals() and final_dataset is not None:
    print("\n🧹 ANÁLISIS: QUÉ SE HIZO CON CADA TIPO DE NaN")
    print("=" * 70)
    
    print("📋 ESTRATEGIAS DE LIMPIEZA APLICADAS:")
    print("-" * 50)
    
    # Estrategias por tipo de columna
    strategies = {
        'Precios (OHLC)': {
            'columnas': ['open', 'high', 'low', 'close'],
            'metodo': 'Forward Fill + Backward Fill',
            'razon': 'Mantiene último precio conocido (ej: viernes → fin de semana)'
        },
        'Volumen': {
            'columnas': ['volume'],
            'metodo': 'Forward Fill para stocks, 0 para fines de semana',
            'razon': 'Stocks no cotizan fines de semana, volumen = último conocido'
        },
        'Metadatos': {
            'columnas': ['asset_name', 'asset_class', 'asset_type'],
            'metodo': 'Rellenado con valores constantes',
            'razon': 'Información fija por activo'
        },
        'Symbol': {
            'columnas': ['symbol'],
            'metodo': 'Puede tener NaN si no se asignó correctamente',
            'razon': 'Verificar si falló la asignación durante descarga'
        }
    }
    
    for categoria, info in strategies.items():
        print(f"\n🔸 {categoria}:")
        print(f"   Columnas: {info['columnas']}")
        print(f"   Método: {info['metodo']}")
        print(f"   Razón: {info['razon']}")
        
        # Verificar estado actual de estas columnas
        for col in info['columnas']:
            if col in final_dataset.columns:
                nan_count = final_dataset[col].isnull().sum()
                status = "✅ SIN NaN" if nan_count == 0 else f"⚠️ {nan_count:,} NaN"
                print(f"     {col}: {status}")
    
    # Casos especiales que requieren atención
    print(f"\n🎯 CASOS QUE REQUIEREN DECISIÓN:")
    
    # Symbol con NaN
    if 'symbol' in final_dataset.columns:
        symbol_nan = final_dataset['symbol'].isnull().sum()
        if symbol_nan > 0:
            print(f"\n📍 SYMBOL CON NaN ({symbol_nan:,} registros):")
            print(f"   OPCIONES:")
            print(f"   A) Rellenar con asset_name (ej: BITCOIN → BITCOIN)")
            print(f"   B) Crear símbolo custom (ej: BITCOIN → BTC)")
            print(f"   C) Dejar NaN (puede causar problemas en análisis)")
            
            # Mostrar sample de NaN en symbol
            sample_symbol_nan = final_dataset[final_dataset['symbol'].isnull()]
            if len(sample_symbol_nan) > 0:
                print(f"   ACTIVOS AFECTADOS:")
                affected_assets = sample_symbol_nan['asset_name'].unique()
                for asset in affected_assets[:5]:  # Mostrar primeros 5
                    count = (sample_symbol_nan['asset_name'] == asset).sum()
                    print(f"     {asset}: {count:,} registros sin símbolo")
    
    # Otros NaN problemáticos
    for col in final_dataset.columns:
        if col not in ['symbol']:  # Ya analizamos symbol
            nan_count = final_dataset[col].isnull().sum()
            if nan_count > 0:
                nan_pct = (nan_count / len(final_dataset)) * 100
                print(f"\n📍 {col.upper()} CON NaN ({nan_count:,} registros, {nan_pct:.2f}%):")
                
                # Mostrar dónde ocurren
                if nan_count < 20:  # Si son pocos, mostrar todos
                    nan_sample = final_dataset[final_dataset[col].isnull()][['date', 'asset_name', col]].head(10)
                    display(nan_sample)
                else:
                    # Si son muchos, mostrar resumen por activo
                    nan_by_asset = final_dataset[final_dataset[col].isnull()]['asset_name'].value_counts()
                    print(f"   Por activo: {nan_by_asset.head().to_dict()}")

else:
    print("❌ Dataset no disponible")


🧹 ANÁLISIS: QUÉ SE HIZO CON CADA TIPO DE NaN
📋 ESTRATEGIAS DE LIMPIEZA APLICADAS:
--------------------------------------------------

🔸 Precios (OHLC):
   Columnas: ['open', 'high', 'low', 'close']
   Método: Forward Fill + Backward Fill
   Razón: Mantiene último precio conocido (ej: viernes → fin de semana)
     open: ✅ SIN NaN
     high: ✅ SIN NaN
     low: ✅ SIN NaN
     close: ✅ SIN NaN

🔸 Volumen:
   Columnas: ['volume']
   Método: Forward Fill para stocks, 0 para fines de semana
   Razón: Stocks no cotizan fines de semana, volumen = último conocido
     volume: ✅ SIN NaN

🔸 Metadatos:
   Columnas: ['asset_name', 'asset_class', 'asset_type']
   Método: Rellenado con valores constantes
   Razón: Información fija por activo
     asset_name: ✅ SIN NaN
     asset_class: ✅ SIN NaN
     asset_type: ✅ SIN NaN

🔸 Symbol:
   Columnas: ['symbol']
   Método: Puede tener NaN si no se asignó correctamente
   Razón: Verificar si falló la asignación durante descarga
     symbol: ⚠️ 14,729 NaN



In [13]:
if 'final_dataset' in globals() and final_dataset is not None:
    print("📊 COMPARACIÓN: ASSET_NAME vs SYMBOL")
    print("=" * 70)
    
    # Verificar que ambas columnas existen
    if 'asset_name' in final_dataset.columns and 'symbol' in final_dataset.columns:
        
        print("🔍 ANÁLISIS DE REDUNDANCIA:")
        print(f"   Total registros: {len(final_dataset):,}")
        print(f"   asset_name únicos: {final_dataset['asset_name'].nunique()}")
        print(f"   symbol únicos: {final_dataset['symbol'].nunique()}")
        print(f"   asset_name NaN: {final_dataset['asset_name'].isnull().sum():,}")
        print(f"   symbol NaN: {final_dataset['symbol'].isnull().sum():,}")
        
        # Crear comparación por activo
        print(f"\n📋 COMPARACIÓN DIRECTA POR ACTIVO:")
        print("-" * 80)
        print(f"{'ASSET_NAME':<15} {'SYMBOL':<15} {'¿Iguales?':<10} {'Registros':<10} {'Observaciones':<20}")
        print("-" * 80)
        
        # Obtener combinaciones únicas de asset_name y symbol
        comparison = final_dataset.groupby(['asset_name', 'symbol']).size().reset_index(name='count')
        
        # También incluir casos donde symbol es NaN
        nan_symbols = final_dataset[final_dataset['symbol'].isnull()]
        if len(nan_symbols) > 0:
            nan_comparison = nan_symbols.groupby('asset_name').size().reset_index(name='count')
            nan_comparison['symbol'] = 'NaN'
            comparison = pd.concat([comparison, nan_comparison], ignore_index=True)
        
        # Mostrar comparación
        for _, row in comparison.sort_values('asset_name').iterrows():
            asset_name = row['asset_name']
            symbol = row['symbol'] if pd.notna(row['symbol']) else 'NaN'
            count = row['count']
            
            # Determinar si son iguales o diferentes
            if symbol == 'NaN':
                iguales = "❌ NaN"
                obs = "Symbol faltante"
            elif asset_name == symbol:
                iguales = "✅ SÍ"
                obs = "Idénticos"
            elif asset_name.lower() in symbol.lower() or symbol.lower() in asset_name.lower():
                iguales = "🟡 Similar"
                obs = "Relacionados"
            else:
                iguales = "❌ NO"
                obs = "Diferentes"
            
            print(f"{asset_name:<15} {symbol:<15} {iguales:<10} {count:<10,} {obs:<20}")
        
        # Análisis de patrones
        print(f"\n📊 ANÁLISIS DE PATRONES:")
        
        # Casos donde asset_name = symbol
        exact_matches = 0
        similar_matches = 0
        different_matches = 0
        nan_matches = 0
        
        for _, row in comparison.iterrows():
            asset_name = row['asset_name']
            symbol = row['symbol'] if pd.notna(row['symbol']) else None
            
            if symbol is None:
                nan_matches += 1
            elif asset_name == symbol:
                exact_matches += 1
            elif asset_name.lower() in symbol.lower() or symbol.lower() in asset_name.lower():
                similar_matches += 1
            else:
                different_matches += 1
        
        total_unique_combinations = len(comparison)
        
        print(f"   ✅ Exactamente iguales: {exact_matches}/{total_unique_combinations}")
        print(f"   🟡 Similares: {similar_matches}/{total_unique_combinations}")
        print(f"   ❌ Diferentes: {different_matches}/{total_unique_combinations}")
        print(f"   ❓ Symbol NaN: {nan_matches}/{total_unique_combinations}")
        
        # Recomendación
        print(f"\n💡 RECOMENDACIÓN:")
        if exact_matches == total_unique_combinations:
            print(f"   🗑️ ELIMINAR 'symbol' - Es completamente redundante")
        elif exact_matches + nan_matches == total_unique_combinations:
            print(f"   🔧 RELLENAR NaN en 'symbol' con asset_name, luego eliminar")
        elif similar_matches > 0 or different_matches > 0:
            print(f"   📊 MANTENER AMBAS - Contienen información diferente")
            print(f"   💭 'asset_name' parece ser el nombre descriptivo")
            print(f"   💭 'symbol' parece ser el ticker técnico")
        else:
            print(f"   🤔 REVISAR MANUALMENTE - Patrones no claros")
        
        # Mostrar ejemplos de diferencias si existen
        if different_matches > 0 or similar_matches > 0:
            print(f"\n📋 EJEMPLOS DE DIFERENCIAS:")
            examples = []
            for _, row in comparison.iterrows():
                asset_name = row['asset_name']
                symbol = row['symbol'] if pd.notna(row['symbol']) else 'NaN'
                
                if asset_name != symbol and symbol != 'NaN':
                    examples.append((asset_name, symbol))
            
            for asset_name, symbol in examples[:5]:  # Mostrar primeros 5
                print(f"     {asset_name} ≠ {symbol}")
        
    else:
        missing_cols = []
        if 'asset_name' not in final_dataset.columns:
            missing_cols.append('asset_name')
        if 'symbol' not in final_dataset.columns:
            missing_cols.append('symbol')
        
        print(f"❌ COLUMNAS FALTANTES: {missing_cols}")
        print(f"📋 COLUMNAS DISPONIBLES: {list(final_dataset.columns)}")

else:
    print("❌ Dataset no disponible")
    print("💡 Ejecuta primero las celdas de descarga de datos")

📊 COMPARACIÓN: ASSET_NAME vs SYMBOL
🔍 ANÁLISIS DE REDUNDANCIA:
   Total registros: 64,838
   asset_name únicos: 17
   symbol únicos: 17
   asset_name NaN: 0
   symbol NaN: 14,729

📋 COMPARACIÓN DIRECTA POR ACTIVO:
--------------------------------------------------------------------------------
ASSET_NAME      SYMBOL          ¿Iguales?  Registros  Observaciones       
--------------------------------------------------------------------------------
AAPL            AAPL            ✅ SÍ       3,269      Idénticos           
AAPL            NaN             ❌ NaN      1,477      Symbol faltante     
AMD             AMD             ✅ SÍ       3,269      Idénticos           
AMD             NaN             ❌ NaN      1,477      Symbol faltante     
AMZN            AMZN            ✅ SÍ       3,269      Idénticos           
AMZN            NaN             ❌ NaN      1,477      Symbol faltante     
BINANCECOIN     BNB-USD         ❌ NO       2,609      Diferentes          
BITCOIN         BTC-USD 

In [15]:
if 'final_dataset' in globals() and final_dataset is not None:
    print("💰 COMPARACIÓN: ASSET_NAME vs SYMBOL - SOLO CRIPTOMONEDAS")
    print("=" * 80)
    
    # Filtrar solo criptomonedas
    if 'asset_class' in final_dataset.columns:
        crypto_data = final_dataset[final_dataset['asset_class'] == 'crypto']
        
        print(f"📊 DATOS DE CRIPTOMONEDAS:")
        print(f"   Total registros crypto: {len(crypto_data):,}")
        print(f"   Criptomonedas únicas: {crypto_data['asset_name'].nunique()}")
        
        if 'asset_name' in crypto_data.columns and 'symbol' in crypto_data.columns:
            
            # Obtener una fila por cada criptomoneda
            crypto_comparison = crypto_data.drop_duplicates('asset_name')[['asset_name', 'symbol']].sort_values('asset_name')
            
            print(f"\n📋 COMPARACIÓN DIRECTA - CRIPTOMONEDAS:")
            print("-" * 60)
            print(f"{'CRYPTO':<15} {'SYMBOL':<15} {'¿Iguales?':<12} {'Diferencia':<15}")
            print("-" * 60)
            
            for _, row in crypto_comparison.iterrows():
                asset_name = row['asset_name']
                symbol = row['symbol'] if pd.notna(row['symbol']) else 'NaN'
                
                # Análisis de similitud
                if symbol == 'NaN':
                    iguales = "❌ NaN"
                    diferencia = "Symbol faltante"
                elif asset_name == symbol:
                    iguales = "✅ IGUALES"
                    diferencia = "Idénticos"
                elif asset_name.upper() in symbol.upper():
                    iguales = "🟡 CONTIENE"
                    diferencia = "Nombre en símbolo"
                elif symbol.upper() in asset_name.upper():
                    iguales = "🟡 CONTIENE"
                    diferencia = "Símbolo en nombre"
                else:
                    iguales = "❌ DIFERENTES"
                    diferencia = "Completamente distinto"
                
                print(f"{asset_name:<15} {symbol:<15} {iguales:<12} {diferencia:<15}")
            
            # Mostrar la tabla completa también
            print(f"\n📊 TABLA COMPLETA - CRIPTOMONEDAS:")
            display(crypto_comparison)
            
            # Análisis específico
            print(f"\n🔍 ANÁLISIS DETALLADO POR CRYPTO:")
            
            for _, row in crypto_comparison.iterrows():
                asset_name = row['asset_name']
                symbol = row['symbol'] if pd.notna(row['symbol']) else None
                
                print(f"\n   💰 {asset_name}:")
                print(f"      Symbol: {symbol if symbol else '❌ NaN'}")
                
                if symbol:
                    # Intentar deducir la relación
                    if 'BITCOIN' in asset_name and 'BTC' in symbol:
                        print(f"      📝 Relación: BITCOIN → BTC-USD (ticker estándar)")
                    elif 'ETHEREUM' in asset_name and 'ETH' in symbol:
                        print(f"      📝 Relación: ETHEREUM → ETH-USD (ticker estándar)")
                    elif asset_name == symbol:
                        print(f"      📝 Relación: Idénticos (redundante)")
                    else:
                        print(f"      📝 Relación: {asset_name} → {symbol}")
                else:
                    print(f"      ⚠️ Problema: Símbolo faltante")
            
            # Verificar si hay NaN en crypto symbols
            crypto_nan_symbols = crypto_data[crypto_data['symbol'].isnull()]
            
            if len(crypto_nan_symbols) > 0:
                print(f"\n🚨 CRIPTOMONEDAS CON SYMBOL NaN:")
                nan_assets = crypto_nan_symbols['asset_name'].unique()
                for asset in nan_assets:
                    count = (crypto_nan_symbols['asset_name'] == asset).sum()
                    total = (crypto_data['asset_name'] == asset).sum()
                    print(f"   {asset}: {count:,}/{total:,} registros sin símbolo")
            
            # Recomendación específica para crypto
            print(f"\n💡 RECOMENDACIÓN PARA CRIPTOMONEDAS:")
            
            # Contar tipos de relaciones
            exact_matches = 0
            related_matches = 0
            different_matches = 0
            nan_matches = 0
            
            for _, row in crypto_comparison.iterrows():
                asset_name = row['asset_name']
                symbol = row['symbol'] if pd.notna(row['symbol']) else None
                
                if symbol is None:
                    nan_matches += 1
                elif asset_name == symbol:
                    exact_matches += 1
                elif (asset_name.upper() in symbol.upper() or 
                      symbol.upper().replace('-USD', '') in asset_name.upper() or
                      any(word in symbol.upper() for word in ['BTC', 'ETH', 'ADA', 'SOL', 'DOT', 'BNB', 'DOGE'])):
                    related_matches += 1
                else:
                    different_matches += 1
            
            print(f"   ✅ Exactos: {exact_matches}")
            print(f"   🟡 Relacionados: {related_matches} (ej: BITCOIN vs BTC-USD)")
            print(f"   ❌ Diferentes: {different_matches}")
            print(f"   ❓ NaN: {nan_matches}")
            
            if related_matches > 0:
                print(f"\n   📊 CONCLUSIÓN CRYPTO:")
                print(f"   • asset_name: Nombre descriptivo (BITCOIN, ETHEREUM)")
                print(f"   • symbol: Ticker técnico para trading (BTC-USD, ETH-USD)")
                print(f"   • AMBAS son ÚTILES - NO eliminar")
                print(f"   • Acción: Solo arreglar NaN si existen")
            elif exact_matches == len(crypto_comparison):
                print(f"   • Las columnas son IDÉNTICAS - considerar eliminar 'symbol'")
            else:
                print(f"   • Patrón mixto - revisar caso por caso")
        
        else:
            print("❌ Columnas asset_name o symbol no encontradas")
    
    else:
        print("❌ Columna asset_class no encontrada - no se pueden filtrar cryptos")

else:
    print("❌ Dataset no disponible")

💰 COMPARACIÓN: ASSET_NAME vs SYMBOL - SOLO CRIPTOMONEDAS
📊 DATOS DE CRIPTOMONEDAS:
   Total registros crypto: 17,514
   Criptomonedas únicas: 7

📋 COMPARACIÓN DIRECTA - CRIPTOMONEDAS:
------------------------------------------------------------
CRYPTO          SYMBOL          ¿Iguales?    Diferencia     
------------------------------------------------------------
BINANCECOIN     BNB-USD         ❌ DIFERENTES Completamente distinto
BITCOIN         BTC-USD         ❌ DIFERENTES Completamente distinto
CARDANO         ADA-USD         ❌ DIFERENTES Completamente distinto
DOGECOIN        DOGE-USD        ❌ DIFERENTES Completamente distinto
ETHEREUM        ETH-USD         ❌ DIFERENTES Completamente distinto
POLKADOT        DOT-USD         ❌ DIFERENTES Completamente distinto
SOLANA          SOL-USD         ❌ DIFERENTES Completamente distinto

📊 TABLA COMPLETA - CRIPTOMONEDAS:


Price,asset_name,symbol
14238,BINANCECOIN,BNB-USD
16847,BITCOIN,BTC-USD
20605,CARDANO,ADA-USD
23214,DOGECOIN,DOGE-USD
25823,ETHEREUM,ETH-USD
52026,POLKADOT,DOT-USD
58366,SOLANA,SOL-USD



🔍 ANÁLISIS DETALLADO POR CRYPTO:

   💰 BINANCECOIN:
      Symbol: BNB-USD
      📝 Relación: BINANCECOIN → BNB-USD

   💰 BITCOIN:
      Symbol: BTC-USD
      📝 Relación: BITCOIN → BTC-USD (ticker estándar)

   💰 CARDANO:
      Symbol: ADA-USD
      📝 Relación: CARDANO → ADA-USD

   💰 DOGECOIN:
      Symbol: DOGE-USD
      📝 Relación: DOGECOIN → DOGE-USD

   💰 ETHEREUM:
      Symbol: ETH-USD
      📝 Relación: ETHEREUM → ETH-USD (ticker estándar)

   💰 POLKADOT:
      Symbol: DOT-USD
      📝 Relación: POLKADOT → DOT-USD

   💰 SOLANA:
      Symbol: SOL-USD
      📝 Relación: SOLANA → SOL-USD

💡 RECOMENDACIÓN PARA CRIPTOMONEDAS:
   ✅ Exactos: 0
   🟡 Relacionados: 7 (ej: BITCOIN vs BTC-USD)
   ❌ Diferentes: 0
   ❓ NaN: 0

   📊 CONCLUSIÓN CRYPTO:
   • asset_name: Nombre descriptivo (BITCOIN, ETHEREUM)
   • symbol: Ticker técnico para trading (BTC-USD, ETH-USD)
   • AMBAS son ÚTILES - NO eliminar
   • Acción: Solo arreglar NaN si existen


In [17]:
if 'final_dataset' in globals() and final_dataset is not None:
    print("\n📈 COMPARACIÓN: ASSET_NAME vs SYMBOL - SOLO ACCIONES")
    print("=" * 80)
    
    # Filtrar solo acciones para comparar
    if 'asset_class' in final_dataset.columns:
        stock_data = final_dataset[final_dataset['asset_class'] == 'stock']
        
        print(f"📊 DATOS DE ACCIONES:")
        print(f"   Total registros stocks: {len(stock_data):,}")
        print(f"   Acciones únicas: {stock_data['asset_name'].nunique()}")
        
        if 'asset_name' in stock_data.columns and 'symbol' in stock_data.columns:
            
            # Obtener una fila por cada acción
            stock_comparison = stock_data.drop_duplicates('asset_name')[['asset_name', 'symbol']].sort_values('asset_name')
            
            print(f"\n📋 COMPARACIÓN - ACCIONES:")
            print("-" * 50)
            print(f"{'STOCK':<15} {'SYMBOL':<10} {'¿Iguales?':<12}")
            print("-" * 50)
            
            for _, row in stock_comparison.iterrows():
                asset_name = row['asset_name']
                symbol = row['symbol'] if pd.notna(row['symbol']) else 'NaN'
                
                if symbol == 'NaN':
                    iguales = "❌ NaN"
                elif asset_name == symbol:
                    iguales = "✅ IGUALES"
                else:
                    iguales = "❌ DIFERENTES"
                
                print(f"{asset_name:<15} {symbol:<10} {iguales:<12}")
            
            print(f"\n📊 TABLA ACCIONES:")
            display(stock_comparison)
            
            print(f"\n💡 PATRÓN EN ACCIONES:")
            print(f"   📊 Las acciones típicamente usan:")
            print(f"   • asset_name: Nombre de empresa (AAPL, TSLA)")
            print(f"   • symbol: Ticker bursátil (AAPL, TSLA)")
            print(f"   • Suelen ser IDÉNTICOS")
    
    print(f"\n🎯 RESUMEN GENERAL:")
    print(f"   💰 CRYPTO: asset_name ≠ symbol (útiles ambos)")
    print(f"   📈 STOCKS: asset_name = symbol (posiblemente redundante)")

else:
    print("❌ Dataset no disponible")


📈 COMPARACIÓN: ASSET_NAME vs SYMBOL - SOLO ACCIONES
📊 DATOS DE ACCIONES:
   Total registros stocks: 47,324
   Acciones únicas: 10

📋 COMPARACIÓN - ACCIONES:
--------------------------------------------------
STOCK           SYMBOL     ¿Iguales?   
--------------------------------------------------
AAPL            AAPL       ✅ IGUALES   
AMD             AMD        ✅ IGUALES   
AMZN            AMZN       ✅ IGUALES   
GOOGL           GOOGL      ✅ IGUALES   
INTC            INTC       ✅ IGUALES   
META            META       ✅ IGUALES   
MSFT            MSFT       ✅ IGUALES   
NVDA            NVDA       ✅ IGUALES   
QCOM            QCOM       ✅ IGUALES   
TSLA            TSLA       ✅ IGUALES   

📊 TABLA ACCIONES:


Price,asset_name,symbol
0,AAPL,AAPL
4746,AMD,AMD
9492,AMZN,AMZN
28432,GOOGL,GOOGL
33178,INTC,INTC
37924,META,META
42534,MSFT,MSFT
47280,NVDA,NVDA
53620,QCOM,QCOM
60092,TSLA,TSLA



💡 PATRÓN EN ACCIONES:
   📊 Las acciones típicamente usan:
   • asset_name: Nombre de empresa (AAPL, TSLA)
   • symbol: Ticker bursátil (AAPL, TSLA)
   • Suelen ser IDÉNTICOS

🎯 RESUMEN GENERAL:
   💰 CRYPTO: asset_name ≠ symbol (útiles ambos)
   📈 STOCKS: asset_name = symbol (posiblemente redundante)


In [18]:
if 'final_dataset' in globals() and final_dataset is not None:
    print("🔧 CORRIGIENDO ASSET_NAME DE STOCKS PARA CONSISTENCIA")
    print("=" * 70)
    
    # Mapeo de correcciones para stocks
    stock_name_corrections = {
        'AAPL': 'Apple',
        'AMD': 'Advanced Micro Devices', 
        'AMZN': 'Amazon',
        'GOOGL': 'Google',
        'INTC': 'Intel',
        'META': 'Meta',  # Ya está bien
        'MSFT': 'Microsoft',
        'NVDA': 'NVIDIA',
        'QCOM': 'Qualcomm',
        'TSLA': 'Tesla'
    }
    
    print("📋 MAPEO DE CORRECCIONES:")
    for ticker, company_name in stock_name_corrections.items():
        current_name = final_dataset[final_dataset['symbol'] == ticker]['asset_name'].iloc[0] if len(final_dataset[final_dataset['symbol'] == ticker]) > 0 else "No encontrado"
        change_needed = "✅ CAMBIAR" if current_name == ticker else "🟢 YA CORRECTO"
        print(f"   {ticker:<6} → {company_name:<25} (actual: {current_name}) {change_needed}")
    
    # Aplicar correcciones
    print(f"\n🔄 APLICANDO CORRECCIONES...")
    corrections_applied = 0
    
    for ticker, company_name in stock_name_corrections.items():
        # Encontrar registros con este símbolo
        mask = final_dataset['symbol'] == ticker
        affected_rows = mask.sum()
        
        if affected_rows > 0:
            current_name = final_dataset.loc[mask, 'asset_name'].iloc[0]
            
            if current_name != company_name:
                # Aplicar corrección
                final_dataset.loc[mask, 'asset_name'] = company_name
                corrections_applied += 1
                print(f"   ✅ {ticker}: {current_name} → {company_name} ({affected_rows:,} registros)")
            else:
                print(f"   🟢 {ticker}: Ya correcto como {company_name}")
        else:
            print(f"   ⚠️ {ticker}: No encontrado en dataset")
    
    print(f"\n📊 RESUMEN DE CORRECCIONES:")
    print(f"   Correcciones aplicadas: {corrections_applied}")
    print(f"   Total empresas: {len(stock_name_corrections)}")
    
else:
    print("❌ Dataset no disponible")

🔧 CORRIGIENDO ASSET_NAME DE STOCKS PARA CONSISTENCIA
📋 MAPEO DE CORRECCIONES:
   AAPL   → Apple                     (actual: AAPL) ✅ CAMBIAR
   AMD    → Advanced Micro Devices    (actual: AMD) ✅ CAMBIAR
   AMZN   → Amazon                    (actual: AMZN) ✅ CAMBIAR
   GOOGL  → Google                    (actual: GOOGL) ✅ CAMBIAR
   INTC   → Intel                     (actual: INTC) ✅ CAMBIAR
   META   → Meta                      (actual: META) ✅ CAMBIAR
   MSFT   → Microsoft                 (actual: MSFT) ✅ CAMBIAR
   NVDA   → NVIDIA                    (actual: NVDA) ✅ CAMBIAR
   QCOM   → Qualcomm                  (actual: QCOM) ✅ CAMBIAR
   TSLA   → Tesla                     (actual: TSLA) ✅ CAMBIAR

🔄 APLICANDO CORRECCIONES...
   ✅ AAPL: AAPL → Apple (3,269 registros)
   ✅ AMD: AMD → Advanced Micro Devices (3,269 registros)
   ✅ AMZN: AMZN → Amazon (3,269 registros)
   ✅ GOOGL: GOOGL → Google (3,269 registros)
   ✅ INTC: INTC → Intel (3,269 registros)
   ✅ META: META → Meta (3,174 regi

In [19]:
if 'final_dataset' in globals() and final_dataset is not None:
    print("\n✅ VERIFICANDO CORRECCIONES APLICADAS")
    print("=" * 60)
    
    # Verificar estructura final
    print("📊 ESTRUCTURA FINAL - CRYPTO vs STOCKS:")
    
    # Mostrar cryptos (deberían mantenerse igual)
    if 'asset_class' in final_dataset.columns:
        crypto_sample = final_dataset[final_dataset['asset_class'] == 'crypto'].drop_duplicates('asset_name')[['asset_name', 'symbol']].sort_values('asset_name')
        
        print(f"\n💰 CRIPTOMONEDAS (sin cambios):")
        print("-" * 40)
        for _, row in crypto_sample.iterrows():
            print(f"   {row['asset_name']:<15} | {row['symbol']}")
        
        # Mostrar stocks (deberían estar corregidos)
        stock_sample = final_dataset[final_dataset['asset_class'] == 'stock'].drop_duplicates('asset_name')[['asset_name', 'symbol']].sort_values('asset_name')
        
        print(f"\n📈 ACCIONES (corregidas):")
        print("-" * 40)
        for _, row in stock_sample.iterrows():
            print(f"   {row['asset_name']:<25} | {row['symbol']}")
        
        # Verificar que ahora son diferentes
        print(f"\n🔍 VERIFICACIÓN DE DIFERENCIACIÓN:")
        
        stocks_different = 0
        stocks_same = 0
        
        for _, row in stock_sample.iterrows():
            if row['asset_name'] != row['symbol']:
                stocks_different += 1
            else:
                stocks_same += 1
        
        print(f"   📈 Stocks con asset_name ≠ symbol: {stocks_different}/{len(stock_sample)}")
        print(f"   📈 Stocks con asset_name = symbol: {stocks_same}/{len(stock_sample)}")
        
        if stocks_different == len(stock_sample):
            print(f"   ✅ PERFECTO: Todas las acciones ahora tienen nombres completos")
        elif stocks_same > 0:
            print(f"   ⚠️ Algunas acciones aún tienen ticker como nombre")
        
        # Mostrar comparación lado a lado
        print(f"\n📊 COMPARACIÓN FINAL LADO A LADO:")
        print("-" * 80)
        print(f"{'CRYPTO':<20} {'SYMBOL':<12} {'STOCK':<25} {'SYMBOL':<8}")
        print("-" * 80)
        
        max_rows = max(len(crypto_sample), len(stock_sample))
        
        for i in range(max_rows):
            crypto_name = crypto_sample.iloc[i]['asset_name'] if i < len(crypto_sample) else ""
            crypto_symbol = crypto_sample.iloc[i]['symbol'] if i < len(crypto_sample) else ""
            stock_name = stock_sample.iloc[i]['asset_name'] if i < len(stock_sample) else ""
            stock_symbol = stock_sample.iloc[i]['symbol'] if i < len(stock_sample) else ""
            
            print(f"{crypto_name:<20} {crypto_symbol:<12} {stock_name:<25} {stock_symbol:<8}")
        
        print(f"\n🎯 RESULTADO FINAL:")
        print(f"   💰 CRYPTO: Nombre descriptivo vs Ticker técnico ✅")
        print(f"   📈 STOCKS: Nombre empresa vs Ticker bursátil ✅")
        print(f"   🔗 CONSISTENCIA: Ambos tipos ahora diferenciados ✅")

else:
    print("❌ Dataset no disponible")


✅ VERIFICANDO CORRECCIONES APLICADAS
📊 ESTRUCTURA FINAL - CRYPTO vs STOCKS:

💰 CRIPTOMONEDAS (sin cambios):
----------------------------------------
   BINANCECOIN     | BNB-USD
   BITCOIN         | BTC-USD
   CARDANO         | ADA-USD
   DOGECOIN        | DOGE-USD
   ETHEREUM        | ETH-USD
   POLKADOT        | DOT-USD
   SOLANA          | SOL-USD

📈 ACCIONES (corregidas):
----------------------------------------
   AAPL                      | nan
   AMD                       | nan
   AMZN                      | nan
   Advanced Micro Devices    | AMD
   Amazon                    | AMZN
   Apple                     | AAPL
   GOOGL                     | nan
   Google                    | GOOGL
   INTC                      | nan
   Intel                     | INTC
   META                      | nan
   MSFT                      | nan
   Meta                      | META
   Microsoft                 | MSFT
   NVDA                      | nan
   NVIDIA                    | NVDA
   QCOM    

In [20]:
if 'final_dataset' in globals() and final_dataset is not None:
    print("🔧 CORRIGIENDO NaN EN COLUMNA SYMBOL")
    print("=" * 60)
    
    # Diagnosticar el problema
    total_nan = final_dataset['symbol'].isnull().sum()
    total_registros = len(final_dataset)
    
    print(f"🚨 DIAGNÓSTICO:")
    print(f"   Total registros: {total_registros:,}")
    print(f"   Symbol NaN: {total_nan:,} ({(total_nan/total_registros)*100:.1f}%)")
    
    if total_nan > 0:
        print(f"\n🔍 NaN POR ACTIVO:")
        nan_by_asset = final_dataset[final_dataset['symbol'].isnull()]['asset_name'].value_counts()
        for asset, count in nan_by_asset.items():
            total_asset = (final_dataset['asset_name'] == asset).sum()
            pct = (count / total_asset) * 100
            print(f"   {asset:<25}: {count:,}/{total_asset:,} ({pct:5.1f}%)")
        
        # Crear mapeo completo asset_name → symbol
        print(f"\n📋 CREANDO MAPEO ASSET_NAME → SYMBOL:")
        
        # Mapeo para criptomonedas
        crypto_symbol_mapping = {
            'BITCOIN': 'BTC-USD',
            'ETHEREUM': 'ETH-USD', 
            'BINANCECOIN': 'BNB-USD',
            'CARDANO': 'ADA-USD',
            'SOLANA': 'SOL-USD',
            'DOGECOIN': 'DOGE-USD',
            'POLKADOT': 'DOT-USD'
        }
        
        # Mapeo para acciones (después de corrección)
        stock_symbol_mapping = {
            'Apple': 'AAPL',
            'Advanced Micro Devices': 'AMD',
            'Amazon': 'AMZN',
            'Google': 'GOOGL',
            'Intel': 'INTC',
            'Meta': 'META',
            'Microsoft': 'MSFT',
            'NVIDIA': 'NVDA',
            'Qualcomm': 'QCOM',
            'Tesla': 'TSLA'
        }
        
        # Combinar mapeos
        complete_mapping = {**crypto_symbol_mapping, **stock_symbol_mapping}
        
        print("💰 CRYPTO MAPPING:")
        for name, symbol in crypto_symbol_mapping.items():
            print(f"   {name:<15} → {symbol}")
        
        print("\n📈 STOCK MAPPING:")
        for name, symbol in stock_symbol_mapping.items():
            print(f"   {name:<25} → {symbol}")
        
        # Aplicar correcciones
        print(f"\n🔄 APLICANDO CORRECCIONES DE SYMBOL:")
        corrections_applied = 0
        
        for asset_name, correct_symbol in complete_mapping.items():
            # Buscar registros de este activo con symbol NaN
            mask = (final_dataset['asset_name'] == asset_name) & (final_dataset['symbol'].isnull())
            affected_rows = mask.sum()
            
            if affected_rows > 0:
                final_dataset.loc[mask, 'symbol'] = correct_symbol
                corrections_applied += affected_rows
                print(f"   ✅ {asset_name}: {affected_rows:,} NaN → {correct_symbol}")
        
        print(f"\n📊 RESUMEN DE CORRECCIONES:")
        print(f"   NaN corregidos: {corrections_applied:,}")
        print(f"   NaN restantes: {final_dataset['symbol'].isnull().sum():,}")
        
    else:
        print("✅ No hay NaN en symbol - no se requiere corrección")

else:
    print("❌ Dataset no disponible")

🔧 CORRIGIENDO NaN EN COLUMNA SYMBOL
🚨 DIAGNÓSTICO:
   Total registros: 64,838
   Symbol NaN: 14,729 (22.7%)

🔍 NaN POR ACTIVO:
   AAPL                     : 1,477/1,477 (100.0%)
   AMD                      : 1,477/1,477 (100.0%)
   AMZN                     : 1,477/1,477 (100.0%)
   GOOGL                    : 1,477/1,477 (100.0%)
   INTC                     : 1,477/1,477 (100.0%)
   MSFT                     : 1,477/1,477 (100.0%)
   QCOM                     : 1,477/1,477 (100.0%)
   NVDA                     : 1,477/1,477 (100.0%)
   TSLA                     : 1,477/1,477 (100.0%)
   META                     : 1,436/1,436 (100.0%)

📋 CREANDO MAPEO ASSET_NAME → SYMBOL:
💰 CRYPTO MAPPING:
   BITCOIN         → BTC-USD
   ETHEREUM        → ETH-USD
   BINANCECOIN     → BNB-USD
   CARDANO         → ADA-USD
   SOLANA          → SOL-USD
   DOGECOIN        → DOGE-USD
   POLKADOT        → DOT-USD

📈 STOCK MAPPING:
   Apple                     → AAPL
   Advanced Micro Devices    → AMD
   Amazon     

In [21]:
if 'final_dataset' in globals() and final_dataset is not None:
    print("\n✅ VERIFICANDO CORRECCIÓN DE SYMBOL COMPLETA")
    print("=" * 60)
    
    # Verificar estado final
    final_nan = final_dataset['symbol'].isnull().sum()
    
    print(f"🔍 VERIFICACIÓN FINAL:")
    print(f"   Symbol NaN restantes: {final_nan:,}")
    
    if final_nan == 0:
        print("   ✅ PERFECTO: Sin NaN en symbol")
    else:
        print("   ⚠️ Aún hay NaN - verificar activos:")
        remaining_nan = final_dataset[final_dataset['symbol'].isnull()]['asset_name'].value_counts()
        for asset, count in remaining_nan.items():
            print(f"     {asset}: {count:,} NaN")
    
    # Mostrar tabla final limpia
    print(f"\n📊 TABLA FINAL: ASSET_NAME vs SYMBOL")
    print("-" * 70)
    print(f"{'ASSET_NAME':<25} {'SYMBOL':<12} {'TIPO':<6} {'REGISTROS':<10}")
    print("-" * 70)
    
    # Obtener resumen por activo
    asset_summary = final_dataset.groupby(['asset_name', 'symbol', 'asset_class']).size().reset_index(name='registros')
    asset_summary = asset_summary.sort_values(['asset_class', 'asset_name'])
    
    for _, row in asset_summary.iterrows():
        asset_name = row['asset_name']
        symbol = row['symbol'] if pd.notna(row['symbol']) else 'NaN'
        asset_type = row['asset_class']
        registros = row['registros']
        
        print(f"{asset_name:<25} {symbol:<12} {asset_type:<6} {registros:<10,}")
    
    # Verificar consistencia
    print(f"\n🎯 VERIFICACIÓN DE CONSISTENCIA:")
    
    # ¿Cada activo tiene un único símbolo?
    multiple_symbols = final_dataset.groupby('asset_name')['symbol'].nunique()
    problematic = multiple_symbols[multiple_symbols > 1]
    
    if len(problematic) == 0:
        print("   ✅ Cada activo tiene exactamente 1 símbolo")
    else:
        print("   ⚠️ Activos con múltiples símbolos:")
        for asset, count in problematic.items():
            print(f"     {asset}: {count} símbolos diferentes")
    
    # ¿Crypto vs Stock patterns correctos?
    crypto_data = final_dataset[final_dataset['asset_class'] == 'crypto']
    stock_data = final_dataset[final_dataset['asset_class'] == 'stock']
    
    print(f"\n💰 PATRONES CRYPTO:")
    crypto_unique = crypto_data.drop_duplicates('asset_name')[['asset_name', 'symbol']]
    for _, row in crypto_unique.iterrows():
        pattern = "✅ Correcto" if row['asset_name'] != row['symbol'] else "⚠️ Iguales"
        print(f"   {row['asset_name']} | {row['symbol']} - {pattern}")
    
    print(f"\n📈 PATRONES STOCK:")
    stock_unique = stock_data.drop_duplicates('asset_name')[['asset_name', 'symbol']]
    for _, row in stock_unique.iterrows():
        pattern = "✅ Correcto" if row['asset_name'] != row['symbol'] else "⚠️ Iguales"
        print(f"   {row['asset_name']} | {row['symbol']} - {pattern}")
    
    total_correct_patterns = 0
    total_assets = len(asset_summary)
    
    for _, row in asset_summary.iterrows():
        if row['asset_name'] != row['symbol']:
            total_correct_patterns += 1
    
    print(f"\n📊 RESUMEN GENERAL:")
    print(f"   Activos con patrón correcto: {total_correct_patterns}/{total_assets}")
    print(f"   Porcentaje éxito: {(total_correct_patterns/total_assets)*100:.1f}%")

else:
    print("❌ Dataset no disponible")


✅ VERIFICANDO CORRECCIÓN DE SYMBOL COMPLETA
🔍 VERIFICACIÓN FINAL:
   Symbol NaN restantes: 14,729
   ⚠️ Aún hay NaN - verificar activos:
     AAPL: 1,477 NaN
     AMD: 1,477 NaN
     AMZN: 1,477 NaN
     GOOGL: 1,477 NaN
     INTC: 1,477 NaN
     MSFT: 1,477 NaN
     QCOM: 1,477 NaN
     NVDA: 1,477 NaN
     TSLA: 1,477 NaN
     META: 1,436 NaN

📊 TABLA FINAL: ASSET_NAME vs SYMBOL
----------------------------------------------------------------------
ASSET_NAME                SYMBOL       TIPO   REGISTROS 
----------------------------------------------------------------------
BINANCECOIN               BNB-USD      crypto 2,609     
BITCOIN                   BTC-USD      crypto 3,758     
CARDANO                   ADA-USD      crypto 2,609     
DOGECOIN                  DOGE-USD     crypto 2,609     
ETHEREUM                  ETH-USD      crypto 2,609     
POLKADOT                  DOT-USD      crypto 1,594     
SOLANA                    SOL-USD      crypto 1,726     
Advanced Micro De

In [9]:
if 'final_dataset' in globals() and final_dataset is not None:
    print("🧹 ELIMINANDO FILAS DUPLICADAS Y CONSOLIDANDO SYMBOL")
    print("=" * 70)
    
    print("🔍 DIAGNÓSTICO DEL PROBLEMA:")
    print(f"   Total registros antes: {len(final_dataset):,}")
    
    # Identificar el problema exacto
    problem_assets = ['AAPL', 'AMD', 'AMZN', 'GOOGL', 'INTC', 'MSFT', 'NVDA', 'QCOM', 'TSLA', 'META']
    
    print("\n📊 IDENTIFICANDO FILAS PROBLEMÁTICAS:")
    for asset in problem_assets:
        # Contar filas con ticker como asset_name
        ticker_rows = (final_dataset['asset_name'] == asset).sum()
        if ticker_rows > 0:
            print(f"   {asset}: {ticker_rows:,} filas con ticker como asset_name")
    
    # SOLUCIÓN: Eliminar filas donde asset_name es un ticker de stock
    print(f"\n🔄 ELIMINANDO FILAS CON TICKER COMO ASSET_NAME...")
    
    # Crear máscara para eliminar filas problemáticas
    ticker_mask = final_dataset['asset_name'].isin(problem_assets)
    rows_to_remove = ticker_mask.sum()
    
    print(f"   Filas a eliminar: {rows_to_remove:,}")
    
    # Aplicar filtro - mantener solo filas que NO tienen ticker como asset_name
    final_dataset_clean = final_dataset[~ticker_mask].copy()
    
    print(f"   Registros después: {len(final_dataset_clean):,}")
    print(f"   Reducción: {len(final_dataset) - len(final_dataset_clean):,} filas")
    
    # Verificar que se solucionó el problema
    remaining_nan = final_dataset_clean['symbol'].isnull().sum()
    print(f"\n✅ VERIFICACIÓN POST-LIMPIEZA:")
    print(f"   Symbol NaN restantes: {remaining_nan:,}")
    
    if remaining_nan == 0:
        print("   🎉 ¡PROBLEMA SOLUCIONADO!")
    else:
        print("   ⚠️ Aún hay NaN - investigar más")
        remaining_assets = final_dataset_clean[final_dataset_clean['symbol'].isnull()]['asset_name'].value_counts()
        for asset, count in remaining_assets.items():
            print(f"     {asset}: {count:,} NaN")
    
    # Mostrar estructura final limpia
    print(f"\n📊 ESTRUCTURA FINAL LIMPIA:")
    clean_summary = final_dataset_clean.groupby(['asset_name', 'symbol', 'asset_class']).size().reset_index(name='registros')
    clean_summary = clean_summary.sort_values(['asset_class', 'asset_name'])
    
    print("-" * 70)
    print(f"{'ASSET_NAME':<25} {'SYMBOL':<12} {'TIPO':<6} {'REGISTROS':<10}")
    print("-" * 70)
    
    for _, row in clean_summary.iterrows():
        print(f"{row['asset_name']:<25} {row['symbol']:<12} {row['asset_class']:<6} {row['registros']:<10,}")
    
    # Actualizar dataset principal
    final_dataset = final_dataset_clean.copy()
    
    print(f"\n📊 RESUMEN FINAL:")
    print(f"   📏 Registros: {len(final_dataset):,}")
    print(f"   🎯 Activos únicos: {final_dataset['asset_name'].nunique()}")
    print(f"   📡 Símbolos únicos: {final_dataset['symbol'].nunique()}")
    print(f"   ❓ Symbol NaN: {final_dataset['symbol'].isnull().sum():,}")
    print(f"   ✅ Dataset limpio: {'SÍ' if final_dataset['symbol'].isnull().sum() == 0 else 'NO'}")
    
    if final_dataset['symbol'].isnull().sum() == 0:
        print(f"\n🎉 ¡ÉXITO TOTAL!")
        print(f"   ✅ Sin filas duplicadas")
        print(f"   ✅ Sin NaN en symbol") 
        print(f"   ✅ Asset names consistentes")
        print(f"   ✅ Listo para análisis")
        
        # Guardar dataset final
        output_file = '../../data/processed/final_crypto_stock_clean.csv'
        final_dataset.to_csv(output_file, index=False)
        file_size = os.path.getsize(output_file) / 1024 / 1024
        
        print(f"\n💾 DATASET FINAL GUARDADO:")
        print(f"   📄 {output_file}")
        print(f"   💽 {file_size:.1f} MB")
        print(f"   📊 {len(final_dataset):,} filas × {len(final_dataset.columns)} columnas")

else:
    print("❌ Dataset no disponible")

🧹 ELIMINANDO FILAS DUPLICADAS Y CONSOLIDANDO SYMBOL
🔍 DIAGNÓSTICO DEL PROBLEMA:
   Total registros antes: 64,838

📊 IDENTIFICANDO FILAS PROBLEMÁTICAS:
   AAPL: 4,746 filas con ticker como asset_name
   AMD: 4,746 filas con ticker como asset_name
   AMZN: 4,746 filas con ticker como asset_name
   GOOGL: 4,746 filas con ticker como asset_name
   INTC: 4,746 filas con ticker como asset_name
   MSFT: 4,746 filas con ticker como asset_name
   NVDA: 4,746 filas con ticker como asset_name
   QCOM: 4,746 filas con ticker como asset_name
   TSLA: 4,746 filas con ticker como asset_name
   META: 4,610 filas con ticker como asset_name

🔄 ELIMINANDO FILAS CON TICKER COMO ASSET_NAME...
   Filas a eliminar: 47,324
   Registros después: 17,514
   Reducción: 47,324 filas

✅ VERIFICACIÓN POST-LIMPIEZA:
   Symbol NaN restantes: 0
   🎉 ¡PROBLEMA SOLUCIONADO!

📊 ESTRUCTURA FINAL LIMPIA:
----------------------------------------------------------------------
ASSET_NAME                SYMBOL       TIPO   REGI

In [25]:
#verificar el tipo actual de la columna volume
print("tipo actual de 'volume':", final_dataset['volume'].dtype)
#convertir la columna de float a integer
final_dataset['volume'] = final_dataset['volume'].astype('Int64')
#verificar el nuevo tipo de la columna volume
print("nuevo tipo de 'volume':", final_dataset['volume'].dtype)
#guardar el dataset final con la columna volume como integer
final_dataset.to_csv('data/processed/final_crypto_stock_clean.csv', index=False)


tipo actual de 'volume': float64
nuevo tipo de 'volume': Int64


In [28]:
#imprimir las primeras filas del dataset final y las ultimas filas del dataset final
print(final_dataset.head())
print(final_dataset.tail())


Price       date      close       high        low       open     volume  \
0     2012-01-03  12.345174  12.383300  12.278229  12.290238  302220800   
1     2012-01-04  12.411519  12.448745  12.286636  12.308250  260022000   
2     2012-01-05  12.549310  12.564920  12.388401  12.456848  271269600   
3     2012-01-06  12.680497  12.691004  12.585033  12.601544  318292800   
6     2012-01-09  12.660385  12.841106  12.648977  12.773562  394024400   

Price asset_name asset_type asset_class symbol  
0          Apple      stock       stock   AAPL  
1          Apple      stock       stock   AAPL  
2          Apple      stock       stock   AAPL  
3          Apple      stock       stock   AAPL  
6          Apple      stock       stock   AAPL  
Price       date       close        high         low        open    volume  \
64830 2024-12-23  430.600006  434.510010  415.410004  431.000000  72698100   
64831 2024-12-24  462.279999  462.779999  435.140015  435.899994  59551800   
64833 2024-12-26  454

In [ ]:

from pathlib import Path

print("BÚSQUEDA ACTUALIZADA CON META/FACEBOOK")
print("="*40)

# Rutas
crypto_path = Path("../../data/raw/crypto")
stocks_path = Path("../../data/raw/stocks")

# Activos con variantes de nombres
activos_buscar = {
    # Stocks
    "aapl": ["aapl"],
    "amd": ["amd"],
    "amzn": ["amzn", "amazon"],
    "googl": ["googl", "google"],
    "intc": ["intc", "intel"],
    "msft": ["msft", "microsoft"],
    "qcom": ["qcom", "qualcomm"],
    "nvda": ["nvda", "nvidia"],
    "tsla": ["tsla", "tesla"],
    "meta": ["meta", "facebook", "fb"],  # ← AQUÍ ESTÁ LA CLAVE
    
    # Crypto
    "bitcoin": ["bitcoin", "btc"],
    "ethereum": ["ethereum", "eth"],
    "binancecoin": ["binancecoin", "bnb", "binance"],
    "cardano": ["cardano", "ada"],
    "dogecoin": ["dogecoin", "doge"],
    "polkadot": ["polkadot", "dot"],
    "solana": ["solana", "sol"]
}

datos = {}
total_filas = 0

def buscar_archivo(activo, variantes, archivos):
    """Busca un archivo que coincida con cualquiera de las variantes"""
    for archivo in archivos:
        nombre = archivo.stem.lower()
        for variante in variantes:
            if variante.lower() in nombre:
                return archivo
    return None

# Buscar en crypto
if crypto_path.exists():
    archivos_crypto = list(crypto_path.glob("*.csv")) + list(crypto_path.glob("*.txt"))
    print(f"\nCrypto: {len(archivos_crypto)} archivos")
    
    for activo, variantes in activos_buscar.items():
        if activo in ["bitcoin", "ethereum", "binancecoin", "cardano", "dogecoin", "polkadot", "solana"]:
            archivo = buscar_archivo(activo, variantes, archivos_crypto)
            if archivo:
                print(f"✅ {activo} → {archivo.name}")
                
                try:
                    df = pd.read_csv(archivo)
                    filas = len(df)
                    total_filas += filas
                    
                    print(f"  Filas: {filas:,}")
                    
                    # Análisis rápido de problemas
                    df.columns = [c.lower().replace(' ', '_') for c in df.columns]
                    
                    for col in ['close', 'volume']:
                        if col in df.columns:
                            zeros = (df[col] == 0).sum()
                            nans = df[col].isna().sum()
                            if zeros > 0 or nans > 0:
                                print(f"  {col}: {zeros} zeros, {nans} NaNs")
                    
                    datos[activo] = df
                    
                except Exception as e:
                    print(f"  Error: {e}")
            else:
                print(f"❌ {activo} no encontrado (buscando: {variantes})")

# Buscar en stocks
if stocks_path.exists():
    archivos_stocks = list(stocks_path.glob("*.csv")) + list(stocks_path.glob("*.txt"))
    print(f"\nStocks: {len(archivos_stocks)} archivos")
    
    for activo, variantes in activos_buscar.items():
        if activo in ["aapl", "amd", "amzn", "googl", "intc", "msft", "qcom", "nvda", "tsla", "meta"]:
            archivo = buscar_archivo(activo, variantes, archivos_stocks)
            if archivo:
                print(f"✅ {activo} → {archivo.name}")
                
                try:
                    df = pd.read_csv(archivo)
                    filas = len(df)
                    total_filas += filas
                    
                    print(f"  Filas: {filas:,}")
                    
                    # Análisis rápido de problemas
                    df.columns = [c.lower().replace(' ', '_') for c in df.columns]
                    
                    for col in ['close', 'volume']:
                        if col in df.columns:
                            zeros = (df[col] == 0).sum()
                            nans = df[col].isna().sum()
                            if zeros > 0 or nans > 0:
                                print(f"  {col}: {zeros} zeros, {nans} NaNs")
                    
                    datos[activo] = df
                    
                except Exception as e:
                    print(f"  Error: {e}")
            else:
                print(f"❌ {activo} no encontrado (buscando: {variantes})")

# RESUMEN FINAL
print(f"\nRESUMEN FINAL:")
print(f"Activos encontrados: {len(datos)}/17")
print(f"Total filas: {total_filas:,}")
print(f"50k+ filas: {'✅ SÍ' if total_filas >= 50000 else '❌ NO'}")

# Mostrar qué se encontró por categoría
crypto_encontrados = [a for a in datos.keys() if a in ["bitcoin", "ethereum", "binancecoin", "cardano", "dogecoin", "polkadot", "solana"]]
stocks_encontrados = [a for a in datos.keys() if a in ["aapl", "amd", "amzn", "googl", "intc", "msft", "qcom", "nvda", "tsla", "meta"]]

print(f"\nDETALLE:")
print(f"Crypto: {len(crypto_encontrados)}/7 - {crypto_encontrados}")
print(f"Stocks: {len(stocks_encontrados)}/10 - {stocks_encontrados}")

# Función para revisar detalles
def revisar_detallado(activo):
    if activo not in datos:
        print(f"❌ {activo} no encontrado")
        print(f"Disponibles: {list(datos.keys())}")
        return
    
    df = datos[activo]
    print(f"\n=== {activo.upper()} - ANÁLISIS DETALLADO ===")
    print(f"Filas totales: {len(df):,}")
    
    # Convertir fecha si existe
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        print(f"Rango fechas: {df['date'].min().date()} a {df['date'].max().date()}")
        
        # Duplicados
        duplicados = df['date'].duplicated().sum()
        if duplicados > 0:
            print(f"⚠️ DUPLICADOS: {duplicados} fechas repetidas")
        else:
            print(f"✅ Sin fechas duplicadas")
    
    # Análisis por columna
    for col in ['close', 'volume']:
        if col in df.columns:
            zeros = df[df[col] == 0]
            nans = df[df[col].isna()]
            
            print(f"\n--- {col.upper()} ---")
            print(f"Zeros: {len(zeros)} ({len(zeros)/len(df)*100:.1f}%)")
            print(f"NaNs: {len(nans)} ({len(nans)/len(df)*100:.1f}%)")
            
            # Mostrar fechas con problemas
            if len(zeros) > 0 and 'date' in df.columns:
                print(f"Fechas con zeros:")
                for _, row in zeros.head(10).iterrows():  # Máximo 10 para no saturar
                    if pd.notna(row.get('date')):
                        fecha = row['date'].strftime('%Y-%m-%d %A')
                        # Clasificar el tipo de día
                        if row['date'].weekday() >= 5:
                            tipo = "[Fin de semana] → Forward fill"
                        elif (row['date'].month == 12 and row['date'].day in [24,25,31]) or (row['date'].month == 1 and row['date'].day == 1):
                            tipo = "[Festivo] → Forward fill"
                        elif row['date'].month == 7 and row['date'].day == 4:
                            tipo = "[4 Julio USA] → Forward fill"
                        else:
                            tipo = "[Día hábil] → ¿Eliminar?"
                        print(f"  {fecha} {tipo}")
                
                if len(zeros) > 10:
                    print(f"  ... y {len(zeros)-10} fechas más con zeros")

print(f"\nPara análisis detallado:")
print(f"revisar_detallado('meta')  # Si se encontró Facebook/Meta")
print(f"revisar_detallado('bitcoin')")
print(f"revisar_detallado('aapl')")

BÚSQUEDA ACTUALIZADA CON META/FACEBOOK

Crypto: 23 archivos
✅ bitcoin → coin_Bitcoin.csv
  Filas: 2,991
  volume: 242 zeros, 0 NaNs
✅ ethereum → coin_Ethereum.csv
  Filas: 2,160
✅ binancecoin → coin_BinanceCoin.csv
  Filas: 1,442
✅ cardano → coin_Cardano.csv
  Filas: 1,374
✅ dogecoin → coin_Dogecoin.csv
  Filas: 2,760
  volume: 11 zeros, 0 NaNs
✅ polkadot → coin_Polkadot.csv
  Filas: 320
✅ solana → coin_Solana.csv
  Filas: 452

Stocks: 7195 archivos
✅ aapl → aapl.us.txt
  Filas: 8,364
  volume: 1 zeros, 0 NaNs
✅ amd → amd.us.txt
  Filas: 8,737
  volume: 1 zeros, 0 NaNs
✅ amzn → amzn.us.txt
  Filas: 5,153
  volume: 1 zeros, 0 NaNs
✅ googl → googl.us.txt
  Filas: 3,333
  volume: 1 zeros, 0 NaNs
✅ intc → intc.us.txt
  Filas: 11,556
  volume: 1 zeros, 0 NaNs
✅ msft → msft.us.txt
  Filas: 7,983
  volume: 1 zeros, 0 NaNs
✅ qcom → qcom.us.txt
  Filas: 6,527
  volume: 1 zeros, 0 NaNs
✅ nvda → nvda.us.txt
  Filas: 4,733
  volume: 1 zeros, 0 NaNs
✅ tsla → tsla.us.txt
  Filas: 1,858
  volume: 1 z

In [7]:
# Ver si los 9 stocks tienen zeros el mismo día (patrón sistemático)
revisar_detallado('aapl')
revisar_detallado('msft') 
revisar_detallado('tsla')

# Ver los 242 zeros de Bitcoin - ¿son fines de semana?
revisar_detallado('bitcoin')


=== AAPL - ANÁLISIS DETALLADO ===
Filas totales: 8,364
Rango fechas: 1984-09-07 a 2017-11-10
✅ Sin fechas duplicadas

--- CLOSE ---
Zeros: 0 (0.0%)
NaNs: 0 (0.0%)

--- VOLUME ---
Zeros: 1 (0.0%)
NaNs: 0 (0.0%)
Fechas con zeros:
  2010-04-26 Monday [Día hábil] → ¿Eliminar?

=== MSFT - ANÁLISIS DETALLADO ===
Filas totales: 7,983
Rango fechas: 1986-03-13 a 2017-11-10
✅ Sin fechas duplicadas

--- CLOSE ---
Zeros: 0 (0.0%)
NaNs: 0 (0.0%)

--- VOLUME ---
Zeros: 1 (0.0%)
NaNs: 0 (0.0%)
Fechas con zeros:
  2010-04-26 Monday [Día hábil] → ¿Eliminar?

=== TSLA - ANÁLISIS DETALLADO ===
Filas totales: 1,858
Rango fechas: 2010-06-28 a 2017-11-10
✅ Sin fechas duplicadas

--- CLOSE ---
Zeros: 0 (0.0%)
NaNs: 0 (0.0%)

--- VOLUME ---
Zeros: 1 (0.1%)
NaNs: 0 (0.0%)
Fechas con zeros:
  2010-06-28 Monday [Día hábil] → ¿Eliminar?

=== BITCOIN - ANÁLISIS DETALLADO ===
Filas totales: 2,991
Rango fechas: 2013-04-29 a 2021-07-06
✅ Sin fechas duplicadas

--- CLOSE ---
Zeros: 0 (0.0%)
NaNs: 0 (0.0%)

--- VOLUME

In [ ]:

from pathlib import Path
import pandas as pd
import numpy as np
print("LIMPIADOR CON FORWARD FILL PARA VOLÚMENES 0")
print("="*45)

# Función para limpiar volumen con forward fill
def limpiar_volumen_forward_fill(df, nombre_activo):
    """Reemplaza volúmenes 0 con el valor del día anterior"""
    
    if 'volume' not in df.columns:
        return df, 0
    
    # Convertir fecha y ordenar
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    
    # Contar zeros antes
    zeros_antes = (df['volume'] == 0).sum()
    
    if zeros_antes > 0:
        print(f"  {nombre_activo}: {zeros_antes} volúmenes con 0 detectados")
        
        # Mostrar primeras fechas con 0
        fechas_zero = df[df['volume'] == 0]['date'].head(3)
        for fecha in fechas_zero:
            print(f"    {fecha.strftime('%Y-%m-%d %A')}")
        if zeros_antes > 3:
            print(f"    ... y {zeros_antes-3} más")
        
        # Aplicar forward fill solo a los valores 0
        # Crear máscara para valores 0
        mask_zeros = df['volume'] == 0
        
        # Reemplazar 0s con NaN temporalmente
        df.loc[mask_zeros, 'volume'] = pd.NA
        
        # Aplicar forward fill (usar valor anterior)
        df['volume'] = df['volume'].fillna(method='ffill')
        
        # Si aún quedan NaN (porque los primeros valores eran 0), usar backward fill
        df['volume'] = df['volume'].fillna(method='bfill')
        
        # Contar cuántos se arreglaron
        zeros_despues = (df['volume'] == 0).sum()
        arreglados = zeros_antes - zeros_despues
        
        print(f"    ✅ {arreglados} valores corregidos con forward fill")
        
        if zeros_despues > 0:
            print(f"    ⚠️ {zeros_despues} valores 0 permanecen")
    
    return df, zeros_antes

# Cargar datos desde las rutas conocidas
crypto_path = Path("../../data/raw/crypto")
stocks_path = Path("../../data/raw/stocks")

# Definir activos y rutas
activos_config = {
    # Crypto
    "bitcoin": ("crypto", "coin_Bitcoin.csv"),
    "dogecoin": ("crypto", "coin_Dogecoin.csv"),
    
    # Stocks (solo los que tienen problemas de volumen)
    "aapl": ("stocks", "aapl.us.txt"),
    "amd": ("stocks", "amd.us.txt"),
    "amzn": ("stocks", "amzn.us.txt"),
    "googl": ("stocks", "googl.us.txt"),
    "intc": ("stocks", "intc.us.txt"),
    "msft": ("stocks", "msft.us.txt"),
    "qcom": ("stocks", "qcom.us.txt"),
    "nvda": ("stocks", "nvda.us.txt"),
    "tsla": ("stocks", "tsla.us.txt"),
    "meta": ("stocks", "afb.us.txt")
}

datos_limpios = {}
resumen_limpieza = {}

for activo, (tipo, archivo) in activos_config.items():
    print(f"\n🔸 PROCESANDO {activo.upper()}")
    
    # Determinar ruta
    if tipo == "crypto":
        ruta_archivo = crypto_path / archivo
    else:
        ruta_archivo = stocks_path / archivo
    
    if not ruta_archivo.exists():
        print(f"  ❌ Archivo no encontrado: {ruta_archivo}")
        continue
    
    try:
        # Cargar archivo
        df = pd.read_csv(ruta_archivo)
        filas_originales = len(df)
        
        # Estandarizar columnas
        df.columns = [c.lower().replace(' ', '_') for c in df.columns]
        
        print(f"  📊 Filas: {filas_originales:,}")
        
        # Verificar rango de fechas
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
            fecha_inicio = df['date'].min()
            fecha_fin = df['date'].max()
            print(f"  📅 Rango: {fecha_inicio.strftime('%Y-%m-%d')} a {fecha_fin.strftime('%Y-%m-%d')}")
            
            # Para Bitcoin, mostrar específicamente la fecha de inicio
            if activo == "bitcoin":
                print(f"  📍 Bitcoin empezó: {fecha_inicio.strftime('%Y-%m-%d')} ({fecha_inicio.strftime('%B %Y')})")
        
        # Aplicar limpieza de volumen
        df_limpio, zeros_corregidos = limpiar_volumen_forward_fill(df, activo)
        
        # Guardar resultado
        datos_limpios[activo] = df_limpio
        resumen_limpieza[activo] = {
            'filas': len(df_limpio),
            'zeros_corregidos': zeros_corregidos,
            'archivo': str(ruta_archivo)
        }
        
    except Exception as e:
        print(f"  ❌ Error procesando {activo}: {e}")

# RESUMEN DE LIMPIEZA
print(f"\n" + "="*50)
print("RESUMEN DE LIMPIEZA APLICADA")
print("="*50)

total_zeros_corregidos = 0
total_filas = 0

for activo, info in resumen_limpieza.items():
    zeros = info['zeros_corregidos']
    filas = info['filas']
    total_zeros_corregidos += zeros
    total_filas += filas
    
    if zeros > 0:
        print(f"✅ {activo.upper()}: {zeros} volúmenes 0 → forward fill ({filas:,} filas)")
    else:
        print(f"✅ {activo.upper()}: Sin problemas de volumen ({filas:,} filas)")

print(f"\n📊 TOTALES:")
print(f"  Activos procesados: {len(resumen_limpieza)}")
print(f"  Total filas: {total_filas:,}")
print(f"  Volúmenes 0 corregidos: {total_zeros_corregidos}")
print(f"  50k+ filas: {'✅ SÍ' if total_filas >= 50000 else '❌ NO'}")

# Función para verificar la limpieza
def verificar_limpieza(activo):
    """Verifica que la limpieza se aplicó correctamente"""
    
    if activo not in datos_limpios:
        print(f"❌ {activo} no encontrado en datos limpios")
        return
    
    df = datos_limpios[activo]
    
    print(f"\n=== VERIFICACIÓN {activo.upper()} ===")
    print(f"Filas: {len(df):,}")
    
    if 'volume' in df.columns:
        zeros_restantes = (df['volume'] == 0).sum()
        nans_restantes = df['volume'].isna().sum()
        
        print(f"Volume:")
        print(f"  Zeros restantes: {zeros_restantes}")
        print(f"  NaNs restantes: {nans_restantes}")
        
        if zeros_restantes == 0 and nans_restantes == 0:
            print(f"  ✅ Limpieza exitosa - sin problemas de volumen")
        else:
            print(f"  ⚠️ Aún hay {zeros_restantes} zeros y {nans_restantes} NaNs")
        
        # Mostrar algunas estadísticas del volumen
        vol_stats = df['volume'].describe()
        print(f"  Estadísticas volumen:")
        print(f"    Min: {vol_stats['min']:,.0f}")
        print(f"    Máx: {vol_stats['max']:,.0f}")
        print(f"    Media: {vol_stats['mean']:,.0f}")

# Función para guardar datos limpios
def guardar_datos_limpios():
    """Guarda los datos limpios en archivos procesados"""

    output_dir = Path("../../data/processed")
    output_dir.mkdir(exist_ok=True)
    
    print(f"\n💾 GUARDANDO DATOS LIMPIOS EN: {output_dir}")
    
    for activo, df in datos_limpios.items():
        archivo_salida = output_dir / f"{activo}_limpio.csv"
        df.to_csv(archivo_salida, index=False)
        print(f"  ✅ {activo} → {archivo_salida}")
    
    print(f"✅ {len(datos_limpios)} archivos guardados")

print(f"\n💡 FUNCIONES DISPONIBLES:")
print(f"verificar_limpieza('bitcoin')  # Ver si Bitcoin quedó limpio")
print(f"verificar_limpieza('aapl')     # Ver si Apple quedó limpio")
print(f"guardar_datos_limpios()        # Guardar archivos limpios")

LIMPIADOR CON FORWARD FILL PARA VOLÚMENES 0

🔸 PROCESANDO BITCOIN
  📊 Filas: 2,991
  📅 Rango: 2013-04-29 a 2021-07-06
  📍 Bitcoin empezó: 2013-04-29 (April 2013)
  bitcoin: 242 volúmenes con 0 detectados
    2013-04-29 Monday
    2013-04-30 Tuesday
    2013-05-01 Wednesday
    ... y 239 más
    ✅ 242 valores corregidos con forward fill

🔸 PROCESANDO DOGECOIN
  📊 Filas: 2,760
  📅 Rango: 2013-12-16 a 2021-07-06
  dogecoin: 11 volúmenes con 0 detectados
    2013-12-16 Monday
    2013-12-17 Tuesday
    2013-12-18 Wednesday
    ... y 8 más
    ✅ 11 valores corregidos con forward fill

🔸 PROCESANDO AAPL
  📊 Filas: 8,364
  📅 Rango: 1984-09-07 a 2017-11-10
  aapl: 1 volúmenes con 0 detectados
    2010-04-26 Monday
    ✅ 1 valores corregidos con forward fill

🔸 PROCESANDO AMD
  📊 Filas: 8,737
  📅 Rango: 1983-03-21 a 2017-11-10
  amd: 1 volúmenes con 0 detectados
    2010-04-26 Monday
    ✅ 1 valores corregidos con forward fill

🔸 PROCESANDO AMZN
  📊 Filas: 5,153
  📅 Rango: 1997-05-16 a 2017-11-

In [3]:
# Verificar que Bitcoin quedó limpio
verificar_limpieza('bitcoin')

# Verificar un stock
verificar_limpieza('aapl')

# Guardar todos los datos limpios
guardar_datos_limpios()


=== VERIFICACIÓN BITCOIN ===
Filas: 2,991
Volume:
  Zeros restantes: 0
  NaNs restantes: 0
  ✅ Limpieza exitosa - sin problemas de volumen
  Estadísticas volumen:
    Min: 2,857,830
    Máx: 350,967,941,479
    Media: 10,910,125,638

=== VERIFICACIÓN AAPL ===
Filas: 8,364
Volume:
  Zeros restantes: 0
  NaNs restantes: 0
  ✅ Limpieza exitosa - sin problemas de volumen
  Estadísticas volumen:
    Min: 668,424
    Máx: 2,069,769,775
    Media: 106,668,160

💾 GUARDANDO DATOS LIMPIOS EN: ..\data\processed
  ✅ bitcoin → ..\data\processed\bitcoin_limpio.csv
  ✅ dogecoin → ..\data\processed\dogecoin_limpio.csv
  ✅ aapl → ..\data\processed\aapl_limpio.csv
  ✅ amd → ..\data\processed\amd_limpio.csv
  ✅ amzn → ..\data\processed\amzn_limpio.csv
  ✅ googl → ..\data\processed\googl_limpio.csv
  ✅ intc → ..\data\processed\intc_limpio.csv
  ✅ msft → ..\data\processed\msft_limpio.csv
  ✅ qcom → ..\data\processed\qcom_limpio.csv
  ✅ nvda → ..\data\processed\nvda_limpio.csv
  ✅ tsla → ..\data\processed\